# Analyse des Avis et Alertes ANSSI avec Enrichissement des CVE

## Étape 1 : Extraction des Flux RSS

In [1]:
# faire "pip install feedparser" si besoin dans le terminal 
import feedparser

def recuperer_bulletins_rss(url):
    rss_feed = feedparser.parse(url)
    bulletins = []
    for entry in rss_feed.entries:
        bulletin = {
            "Titre du bulletin (ANSSI)": entry.title,
            "Description": entry.description,
            "Lien du bulletin (ANSSI)": entry.link,
            "Date de publication": entry.published
        }
        bulletins.append(bulletin)
    return bulletins

# URLs des flux RSS
avis_url = "https://www.cert.ssi.gouv.fr/avis/feed/"
alerte_url = "https://www.cert.ssi.gouv.fr/alerte/feed/"

avis_bulletins = recuperer_bulletins_rss(avis_url)
alerte_bulletins = recuperer_bulletins_rss(alerte_url)

bulletins = avis_bulletins + alerte_bulletins

# Afficher les bulletins pour vérification
for bulletin in bulletins:
    print(bulletin)

{'Titre du bulletin (ANSSI)': 'Multiples vulnérabilités dans Mattermost Server (13 mai 2025)', 'Description': 'De multiples vulnérabilités ont été découvertes dans Mattermost Server. Elles permettent à un attaquant de provoquer une atteinte à la confidentialité des données et un contournement de la politique de sécurité.', 'Lien du bulletin (ANSSI)': 'https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-AVI-0392/', 'Date de publication': 'Tue, 13 May 2025 00:00:00 +0000'}
{'Titre du bulletin (ANSSI)': 'Multiples vulnérabilités dans les produits SAP (13 mai 2025)', 'Description': "De multiples vulnérabilités ont été découvertes dans les produits SAP. Certaines d'entre elles permettent à un attaquant de provoquer une exécution de code arbitraire à distance, une atteinte à la confidentialité des données et une injection de code indirecte à distance (XSS).", 'Lien du bulletin (ANSSI)': 'https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-AVI-0396/', 'Date de publication': 'Tue, 13 May 2025 00:00:00 +00

## Étape 2 : Extraction des CVE

In [2]:
import requests
import re

def extraire_cves_depuis_bulletin(lien_bulletin):
    # Construit l'URL JSON à partir du lien du bulletin
    if not lien_bulletin.endswith('/'):
        lien_bulletin += '/'
    url_json = lien_bulletin + "json/"
    response = requests.get(url_json)
    data = response.json()
    # Extraction des CVE référencés dans la clé "cves"
    ref_cves = [cve.get("name") for cve in data.get("cves", [])]
    #attention il s’agit d’une liste des dictionnaires avec name et url comme clés
    print( "CVE référencés ", ref_cves)
    # Extraction des CVE avec une regex
    cve_pattern = r"CVE-\d{4}-\d{4,7}"
    cve_list = list(set(re.findall(cve_pattern, str(data))))
    print("CVE trouvés :", cve_list)
    return ref_cves, cve_list

tous_les_cves = set()
for bulletin in bulletins:
    ref_cves, cve_list = extraire_cves_depuis_bulletin(bulletin['Lien du bulletin (ANSSI)'])
    tous_les_cves.update(ref_cves)
    tous_les_cves.update(cve_list)
    bulletin["CVEs"] = list(set(ref_cves + cve_list))
    print(f"Bulletin {bulletin['Titre du bulletin (ANSSI)']} : CVEs = {bulletin['CVEs']}")




CVE référencés  ['CVE-2025-4128', 'CVE-2025-4573']
CVE trouvés : ['CVE-2025-4573', 'CVE-2025-4128']
Bulletin Multiples vulnérabilités dans Mattermost Server (13 mai 2025) : CVEs = ['CVE-2025-4573', 'CVE-2025-4128']
CVE référencés  ['CVE-2025-43003', 'CVE-2025-43007', 'CVE-2025-23191', 'CVE-2025-42999', 'CVE-2025-43009', 'CVE-2025-43011', 'CVE-2025-43006', 'CVE-2025-0060', 'CVE-2025-30012', 'CVE-2025-43000', 'CVE-2025-43004', 'CVE-2025-31324', 'CVE-2025-43005', 'CVE-2025-43008', 'CVE-2025-31329', 'CVE-2025-30009', 'CVE-2025-30011', 'CVE-2025-43002', 'CVE-2025-26662', 'CVE-2025-30010', 'CVE-2025-42997', 'CVE-2025-0061', 'CVE-2025-43010', 'CVE-2024-39592', 'CVE-2025-30018']
CVE trouvés : ['CVE-2025-31329', 'CVE-2025-30012', 'CVE-2025-30010', 'CVE-2025-0061', 'CVE-2025-43005', 'CVE-2025-42999', 'CVE-2025-26662', 'CVE-2024-39592', 'CVE-2025-43006', 'CVE-2025-43003', 'CVE-2025-30018', 'CVE-2025-30011', 'CVE-2025-43010', 'CVE-2025-23191', 'CVE-2025-43004', 'CVE-2025-0060', 'CVE-2025-43008', '

In [3]:
print("Tous les CVEs extraits :", tous_les_cves)

Tous les CVEs extraits : {'CVE-2023-53119', 'CVE-2025-32335', 'CVE-2025-29876', 'CVE-2022-49801', 'CVE-2024-57807', 'CVE-2023-42115', 'CVE-2025-21724', 'CVE-2024-56751', 'CVE-2025-21961', 'CVE-2024-58051', 'CVE-2025-21718', 'CVE-2025-32996', 'CVE-2025-37823', 'CVE-2025-21750', 'CVE-2024-57979', 'CVE-2025-22108', 'CVE-2025-21742', 'CVE-2023-7028', 'CVE-2024-50115', 'CVE-2023-35036', 'CVE-2025-5195', 'CVE-2025-47163', 'CVE-2024-41077', 'CVE-2025-37780', 'CVE-2025-29885', 'CVE-2024-20359', 'CVE-2025-21831', 'CVE-2023-5363', 'CVE-2024-56171', 'CVE-2018-14719', 'CVE-2025-20286', 'CVE-2025-42983', 'CVE-2022-49769', 'CVE-2025-21857', 'CVE-2024-58068', 'CVE-2025-23129', 'CVE-2025-21847', 'CVE-2024-6345', 'CVE-2025-21925', 'CVE-2024-58058', 'CVE-2024-58019', 'CVE-2025-21927', 'CVE-2024-49994', 'CVE-2024-57980', 'CVE-2025-23160', 'CVE-2022-49818', 'CVE-2024-38606', 'CVE-2024-54458', 'CVE-2024-56558', 'CVE-2025-21743', 'CVE-2025-21917', 'CVE-2020-9548', 'CVE-2019-14892', 'CVE-2022-49887', 'CVE-20

## Étape 3 : Enrichissement des CVE

### Exemple de connexion à l'API CVE :

In [4]:
import requests

def extraire_infos_cve(cve_ids):
    resultats = []

    for cve_id in cve_ids:
        url = f"https://cveawg.mitre.org/api/cve/{cve_id}"
        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()

            # Description
            description = (
                data.get("containers", {})
                    .get("cna", {})
                    .get("descriptions", [{}])[0]
                    .get("value", "Non disponible")
            )

            # Score CVSS et Base Severity
            cvss_score = "Non disponible"
            base_severity = "Non disponible"
            metrics = data.get("containers", {}).get("cna", {}).get("metrics", [])
            for metric in metrics:
                for key in ["cvssV3_1", "cvssV3_0", "cvssV2_0"]:
                    if key in metric:
                        score = metric[key].get("baseScore")
                        severity = metric[key].get("baseSeverity", "Non disponible")
                        if score is not None:
                            cvss_score = score
                        if severity is not None:
                            base_severity = severity
                        break
                if cvss_score != "Non disponible" or base_severity != "Non disponible":
                    break

            # CWE
            cwe = "Non disponible"
            cwe_desc = "Non disponible"
            problemtype = data.get("containers", {}).get("cna", {}).get("problemTypes", [])
            if problemtype:
                descriptions = problemtype[0].get("descriptions", [])
                if descriptions:
                    cwe = descriptions[0].get("cweId", "Non disponible")
                    cwe_desc = descriptions[0].get("description", "Non disponible")

            # Produits affectés
            produits = []
            for product in data.get("containers", {}).get("cna", {}).get("affected", []):
                vendor = product.get("vendor", "Non disponible")
                product_name = product.get("product", "Non disponible")
                versions = [
                    v.get("version", "Inconnue")
                    for v in product.get("versions", [])
                    if v.get("status") == "affected"
                ]
                produits.append({
                    "Éditeur": vendor,
                    "Produit": product_name,
                    "Versions": versions
                })

            # Affichage direct ici
            print(f"CVE : {cve_id}")
            print(f"Description : {description}")
            print(f"Score CVSS : {cvss_score}")
            print(f"Base Severity : {base_severity}")
            print(f"Type CWE : {cwe}")
            print(f"CWE Description : {cwe_desc}")
            for produit in produits:
                print(f"Éditeur : {produit['Éditeur']}, Produit : {produit['Produit']}, Versions : {', '.join(produit['Versions'])}")
            print("-" * 40)

            # Stockage des résultats
            resultats.append({
                "CVE": cve_id,
                "Description": description,
                "Score CVSS": cvss_score,
                "Base Severity": base_severity,
                "Type CWE": cwe,
                "CWE Description": cwe_desc,
                "Produits affectés": produits
            })

        except Exception as e:
            print(f"[Erreur] {cve_id} : {e}")

    return resultats

infos_cve = extraire_infos_cve(tous_les_cves)

CVE : CVE-2023-53119
Description : In the Linux kernel, the following vulnerability has been resolved:

nfc: pn533: initialize struct pn533_out_arg properly

struct pn533_out_arg used as a temporary context for out_urb is not
initialized properly. Its uninitialized 'phy' field can be dereferenced in
error cases inside pn533_out_complete() callback function. It causes the
following failure:

general protection fault, probably for non-canonical address 0xdffffc0000000000: 0000 [#1] PREEMPT SMP KASAN
KASAN: null-ptr-deref in range [0x0000000000000000-0x0000000000000007]
CPU: 1 PID: 0 Comm: swapper/1 Not tainted 6.2.0-rc3-next-20230110-syzkaller #0
Hardware name: Google Google Compute Engine/Google Compute Engine, BIOS Google 10/26/2022
RIP: 0010:pn533_out_complete.cold+0x15/0x44 drivers/nfc/pn533/usb.c:441
Call Trace:
 <IRQ>
 __usb_hcd_giveback_urb+0x2b6/0x5c0 drivers/usb/core/hcd.c:1671
 usb_hcd_giveback_urb+0x384/0x430 drivers/usb/core/hcd.c:1754
 dummy_timer+0x1203/0x32d0 drivers/usb/g

### Exemple de connexion à l’API EPSS:

In [5]:
#import requests
def extraire_epss_pour_cves(cve_ids):
    epss_scores = {}
    # URL de l'API EPSS pour récupérer la probabilité d'exploitation
    for cve_id in cve_ids:
        url = f"https://api.first.org/data/v1/epss?cve={cve_id}"
        try:
            # Requête GET pour récupérer les données JSON
            response = requests.get(url)
            data = response.json()
            # Extraire le score EPSS
            epss_data = data.get("data", [])
            if epss_data:
                epss_score = epss_data[0].get("epss")
                epss_scores[cve_id] = epss_score
                print(f"CVE : {cve_id}")
                print(f"Score EPSS : {epss_score}")
            else:
                epss_scores[cve_id] = None
                print(f"Aucun score EPSS trouvé pour {cve_id}")
        except Exception as e:
            epss_scores[cve_id] = None
            print(f"Erreur pour {cve_id} : {e}")
    return epss_scores

epss_scores = extraire_epss_pour_cves(tous_les_cves)


CVE : CVE-2023-53119
Score EPSS : 0.000350000
Aucun score EPSS trouvé pour CVE-2025-32335
CVE : CVE-2025-29876
Score EPSS : 0.001180000
CVE : CVE-2022-49801
Score EPSS : 0.000240000
CVE : CVE-2024-57807
Score EPSS : 0.000140000
CVE : CVE-2023-42115
Score EPSS : 0.637980000
CVE : CVE-2025-21724
Score EPSS : 0.000440000
CVE : CVE-2024-56751
Score EPSS : 0.000270000
CVE : CVE-2025-21961
Score EPSS : 0.000170000
CVE : CVE-2024-58051
Score EPSS : 0.001000000
CVE : CVE-2025-21718
Score EPSS : 0.000200000
CVE : CVE-2025-32996
Score EPSS : 0.000560000
CVE : CVE-2025-37823
Score EPSS : 0.000350000
CVE : CVE-2025-21750
Score EPSS : 0.000330000
CVE : CVE-2024-57979
Score EPSS : 0.000350000
CVE : CVE-2025-22108
Score EPSS : 0.000260000
CVE : CVE-2025-21742
Score EPSS : 0.000210000
CVE : CVE-2023-7028
Score EPSS : 0.940040000
CVE : CVE-2024-50115
Score EPSS : 0.000410000
CVE : CVE-2023-35036
Score EPSS : 0.348940000
CVE : CVE-2025-5195
Score EPSS : 0.000110000
CVE : CVE-2025-47163
Score EPSS : 0.00

## Étape 4 : Consolidation des Données

In [6]:
import pandas as pd

# Supposons que tu as déjà exécuté les fonctions suivantes et stocké les résultats dans ces variables :
# bulletins, ref_cves, cve_list, infos_cve, epss_scores

# bulletins = [...]      # Liste de bulletins (dictionnaires)
# ref_cves = [...]       # Liste des CVE référencés (extraites)
# cve_list = [...]       # Liste des CVE trouvés par regex
# infos_cve = [...]      # Liste des infos enrichies pour chaque CVE
# epss_scores = {...}    # Dictionnaire {cve: score}


infos_cve_dict = {info["CVE"]: info for info in infos_cve}

donnees = []
for bulletin in bulletins:
    cves_bulletin = bulletin.get("CVEs", [])
    if not cves_bulletin:
        # ligne par défaut
        donnees.append({
            "ID du bulletin (ANSSI)": bulletin.get("ID du bulletin (ANSSI)", ""),
            "Titre du bulletin (ANSSI)": bulletin.get("Titre du bulletin (ANSSI)", ""),
            "Type de bulletin": bulletin.get("Type de bulletin", ""),
            "Date de publication": bulletin.get("Date de publication", ""),
            "Identifiant CVE": None,
            "Score CVSS": None,
            "Base Severity": None,
            "Type CWE": None,
            "Score EPSS": None,
            "Lien du bulletin (ANSSI)": bulletin.get("Lien du bulletin (ANSSI)", ""),
            "Description": bulletin.get("Description", ""),
            "Éditeur/Vendor": None,
            "Produit": None,
            "Versions affectées": None
        })
    else:
        for cve in cves_bulletin:
            # enrichissement
            info = infos_cve_dict.get(cve, {})
            produits = info.get("Produits affectés", [{}])
            produit = produits[0] if produits else {}
            donnees.append({
                "ID du bulletin (ANSSI)": bulletin.get("ID du bulletin (ANSSI)", ""),
                "Titre du bulletin (ANSSI)": bulletin.get("Titre du bulletin (ANSSI)", ""),
                "Type de bulletin": bulletin.get("Type de bulletin", ""),
                "Date de publication": bulletin.get("Date de publication", ""),
                "Identifiant CVE": cve,
                "Score CVSS": info.get("Score CVSS"),
                "Base Severity": info.get("Base Severity"),                
                "Type CWE": info.get("Type CWE"),
                "Score EPSS": epss_scores.get(cve),
                "Lien du bulletin (ANSSI)": bulletin.get("Lien du bulletin (ANSSI)", ""),
                "Description": info.get("Description", bulletin.get("Description", "")),
                "Éditeur/Vendor": produit.get("Éditeur"),
                "Produit": produit.get("Produit"),
                "Versions affectées": ", ".join(produit.get("Versions", [])) if produit.get("Versions") else None
            })

df = pd.DataFrame(donnees)
df


,ID du bulletin (ANSSI),Titre du bulletin (ANSSI),Type de bulletin,Date de publication,Identifiant CVE,Score CVSS,Base Severity,Type CWE,Score EPSS,Lien du bulletin (ANSSI),Description,Éditeur/Vendor,Produit,Versions affectées
0,,Multiples vulnérabilités dans Mattermost Serve...,,"Tue, 13 May 2025 00:00:00 +0000",CVE-2025-4573,4.1,MEDIUM,CWE-90,0.000220000,https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-...,"Mattermost versions 10.7.x <= 10.7.1, 10.6.x <...",Mattermost,Mattermost,"10.7.0, 10.6.0, 10.5.0, 9.11.0"
1,,Multiples vulnérabilités dans Mattermost Serve...,,"Tue, 13 May 2025 00:00:00 +0000",CVE-2025-4128,3.1,LOW,CWE-863,0.000210000,https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-...,"Mattermost versions 10.5.x <= 10.5.4, 9.11.x <...",Mattermost,Mattermost,"10.5.0, 9.11.0"
2,,Multiples vulnérabilités dans les produits SAP...,,"Tue, 13 May 2025 00:00:00 +0000",CVE-2025-31329,6.2,MEDIUM,CWE-141,0.000390000,https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-...,SAP NetWeaver is vulnerable to an Information ...,SAP_SE,SAP NetWeaver Application Server ABAP and ABAP...,"SAP_BASIS 700, SAP_BASIS 701, SAP_BASIS 702, S..."
3,,Multiples vulnérabilités dans les produits SAP...,,"Tue, 13 May 2025 00:00:00 +0000",CVE-2025-30012,3.9,LOW,CWE-502,0.000480000,https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-...,The Live Auction Cockpit in SAP Supplier Relat...,SAP_SE,SAP Supplier Relationship Management (Live Auc...,SRM_SERVER 7.14
4,,Multiples vulnérabilités dans les produits SAP...,,"Tue, 13 May 2025 00:00:00 +0000",CVE-2025-30010,6.1,MEDIUM,CWE-601,0.000810000,https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-...,The Live Auction Cockpit in SAP Supplier Relat...,SAP_SE,SAP Supplier Relationship Management (Live Auc...,SRM_SERVER 7.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1283,,Vulnérabilité dans SAP NetWeaver (28 avril 2025),,"Mon, 28 Apr 2025 00:00:00 +0000",CVE-2025-31324,10,CRITICAL,CWE-434,0.744460000,https://www.cert.ssi.gouv.fr/alerte/CERTFR-202...,SAP NetWeaver Visual Composer Metadata Uploade...,SAP_SE,SAP NetWeaver (Visual Composer development ser...,VCFRAMEWORK 7.50
1284,,Vulnérabilité dans les produits Fortinet (13 m...,,"Tue, 13 May 2025 00:00:00 +0000",CVE-2025-32756,9.6,CRITICAL,CWE-121,0.107360000,https://www.cert.ssi.gouv.fr/alerte/CERTFR-202...,A stack-based buffer overflow vulnerability [C...,Fortinet,FortiVoice,"7.2.0, 7.0.0, 6.4.0"
1285,,Multiples vulnérabilités dans Ivanti Endpoint ...,,"Wed, 14 May 2025 00:00:00 +0000",CVE-2025-4427,5.3,MEDIUM,CWE-288,0.801990000,https://www.cert.ssi.gouv.fr/alerte/CERTFR-202...,An authentication bypass in the API component ...,Ivanti,Endpoint Manager Mobile,None
1286,,Multiples vulnérabilités dans Ivanti Endpoint ...,,"Wed, 14 May 2025 00:00:00 +0000",CVE-2025-4428,7.2,HIGH,CWE-94,0.155390000,https://www.cert.ssi.gouv.fr/alerte/CERTFR-202...,Remote Code Execution in API component in Ivan...,Ivanti,Endpoint Manager Mobile,None


In [7]:
df.to_csv("data.csv", index=False, encoding='utf-8', sep=';')